# แก้ไขรายชื่อผู้ทรงคุณวุฒิที่ เสนอสภามหาวิทยาลัย
เปิดไฟล์ Excel ข้อมูลของ ที่ เสนอสภามหาวิทยาลัย (ให้แก้ชื่อคอลัมน์ในไฟล์ Excel ให้ถูกต้องก่อนรันโค้ดนี้) และตั้งชื่อไฟล์สำหรับเก็บข้อมูลที่มีปัญหา

In [1]:
import os
import pandas as pd
from fixblank import rmblankname
from fixwords import fixranks, fixphd, fixuni, fixstatus
from fixnames import fix_name
from fixtitle import fix_title
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 10)
fn = 'รายชื่อผู้ทรงคุณวุฒิ_เสนอสภามหาวิทยาลัย.xlsx'
b = pd.ExcelFile(fn)

กำหนดคอลัมน์ที่มีข้อมูลที่จำเป็น

In [2]:
cols = ['ตำแหน่ง', 'ชื่อ', 'นามสกุล', 'คุณวุฒิ', 'สาขาวิชา',
        'ความเชี่ยวชาญ', 'สถานที่ติดต่อ', 'โทร.', 'email']

สร้างฟังก์ชันแก้ไขตำแหน่ง

In [3]:
def splitpos(s):
    s['ตำแหน่งใหม่'] = s['ตำแหน่ง'].strip()
    if s['ตำแหน่งใหม่'].endswith('ดร.'):
        s['ตำแหน่ง ป.เอก'] = 'ดร.'
        s['ตำแหน่งใหม่'] = s['ตำแหน่ง'][:-3]
    else:
        s['ตำแหน่ง ป.เอก'] = ''    
    return s

# dfs['ตำแหน่ง'] = dfs['ตำแหน่ง'].fillna('')
# mydfs = dfs.apply(splitpos, axis=1)
# print(mydfs[['ตำแหน่งใหม่', 'ตำแหน่ง ป.เอก']])

สร้างฟังก์ชันหาชื่อมหาวิทยาลัยจากสถานที่ติดต่อ

In [4]:
from fixwords import uni

def splituni(s):
    place = ''.join(s.split())
    keys = uni.keys()
    match_pos = [(place.find(x), x) for x in keys if place.find(x) >= 0]
    if not match_pos:
        return ''
    else:
        university = min(match_pos, key = lambda t: t[0])
        return university[1]

# dd = dfs['สถานที่ติดต่อ'].fillna('').apply(splituni)

# for i, x in enumerate(dd):
#     if len(x) != 1:
#         print(x)
#         print(i)
#         print(dfs['สถานที่ติดต่อ'][i])
#     if not x:
#         print(i)
#         print(dfs['สถานที่ติดต่อ'][i])

### รวมข้อมูลจากทุก Sheet

ในไฟล์ Excel ข้อมูลในแต่ละ Sheet แบ่งตามสาขาวิชาของผู้ทรงคุณวุฒิ

ในแต่ละ Sheet ทำดังนี้
* ลบแถวที่ไม่มีข้อมูล
* แยกข้อมูลที่มีปัญหาออกมา และบันทึกในไฟล์ที่กำหนด
* แก้ไขคอลัมน์ คุณวุฒิ และความเชี่ยวชาญ
* เพิ่มคอลัมน์ สาขาวิชา ตามชื่อ Sheet

In [5]:
dfs = None
for c in b.sheet_names:
    print(c)
    d = b.parse(c, skiprows=1)
    d = d.dropna(how='all')
    d.columns = [c.strip() for c in d.columns]
    
    d = d[cols]
    d = rmblankname(d, cols[1:3])
    d['สถานภาพ'] = ''
    
    d['ตำแหน่ง'] = d['ตำแหน่ง'].fillna('')
    d = d.apply(splitpos, axis=1)
    
    d['มหาลัย'] = d['สถานที่ติดต่อ'].fillna('').apply(splituni)

    dfs = pd.concat([dfs, d])
#dfs


Reader


เรียงคอลัมน์ใหม่ และเปลี่ยนชื่อคอลัมน์ใหม่

In [6]:
dfs_cols = ['ตำแหน่งใหม่', 'ตำแหน่ง ป.เอก', 'ชื่อ', 'นามสกุล', 'คุณวุฒิ', 'มหาลัย', 'สาขาวิชา', 'ความเชี่ยวชาญ', 'สถานที่ติดต่อ', 'โทร.', 'email', 'สถานภาพ']
new_cols = ['ตำแหน่ง', 'ตำแหน่ง ป.เอก', 'ชื่อ', 'นามสกุล', 'คุณวุฒิ', 'มหาวิทยาลัย', 'สาขาวิชา', 'ความเชี่ยวชาญ', 'สถานที่ติดต่อ', 'โทร', 'email', 'สถานะ']
dfs2 = dfs[dfs_cols]
dfs2.columns = new_cols

### การแก้ไขข้อมูล
* แก้ชื่อตำแหน่ง, ตำแหน่ง ป.เอก, ชื่อมหาวิทยาลัย ที่ปนระหว่างชื่อเต็ม และตัวย่อ หรือสะกดไม่เหมือนกัน
* แก้ สถานะ ให้มีแค่ เกษียณ, ลาออก, หรือปกติ
* แก้ชื่อ โดยการลบคำนำหน้าชื่อ ทั้งที่เป็นตัวย่อ และคำเต็ม เช่น นาย, นาง, นางสาว

In [7]:
dfs2['ตำแหน่ง'] = dfs2['ตำแหน่ง'].fillna('').apply(fixranks)
dfs2['ตำแหน่ง ป.เอก'] = dfs2['ตำแหน่ง ป.เอก'].fillna('').apply(fixphd)
dfs2['มหาวิทยาลัย'] = dfs2['มหาวิทยาลัย'].fillna('').apply(fixuni)
dfs2['สถานะ'] = dfs2['สถานะ'].fillna('').apply(fixstatus)
dfs2['ชื่อ'] = dfs2['ชื่อ'].fillna('').apply(fix_name)
dfs2['ชื่อ'] = dfs2['ชื่อ'].fillna('').apply(fix_title)

In [8]:
dfs2

,ตำแหน่ง,ตำแหน่ง ป.เอก,ชื่อ,นามสกุล,คุณวุฒิ,มหาวิทยาลัย,สาขาวิชา,ความเชี่ยวชาญ,สถานที่ติดต่อ,โทร,email,สถานะ
0,รองศาสตราจารย์,ดร.,สรณัฐ,ไตลังคะ,อักษรศาสตรดุษฎีบัณฑิต (วรรณคดีและวรรณคดีเปรียบ...,มหาวิทยาลัยเกษตรศาสตร์,วรรณคดี,วรรณกรรมไทยสมัยใหม่ วรรณคดีเปรียบเทียบ วรรณคดี...,ภาควิชาวรรณคดี \nคณะมนุษยศาสตร์ \nมหาวิทยาลัยเ...,02-579-5566,fhumira@ku.ac.th,
1,รองศาสตราจารย์,ดร.,วิภาวรรณ,อยู่เย็น,NaN,มหาวิทยาลัยเกษตรศาสตร์,ภาษาไทย,NaN,ภาควิชาภาษาไทย \nคณะมนุษยศาสตร์ \nมหาวิทยาลัยเ...,NaN,NaN,
2,รองศาสตราจารย์,ดร.,ณัฐมา,พงศ์ไพโรจน์,Ph.D. (Linguistics),จุฬาลงกรณ์มหาวิทยาลัย,ภาษาอังกฤษ,"Second Language Acquisition, especially of Syn...",ภาควิชาภาษาอังกฤษ \nคณะอักษรศาสตร์ \nจุฬาลงกรณ...,02-218-4716,pnattama@gmail.com,
3,รองศาสตราจารย์,,อุดมศักดิ์,สาริบุตร,NaN,สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง,ครุศาสตร์สถาปัตยกรรมและการออกแบบ,NaN,คณะครุศาสตร์อุตสาหกรรม \nสถาบันเทคโนโลยีพระจอม...,NaN,NaN,
4,รองศาสตราจารย์,,สัญญา,วงศ์อร่าม,NaN,จุฬาลงกรณ์มหาวิทยาลัย,ศิลปศึกษา,NaN,คณะครุศาสตร์ \nจุฬาลงกรณ์มหาวิทยาลัย \nแขวงวัง...,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
103,รองศาสตราจารย์,ดร.,กนกพร,นาคทับที,Ph.D. (Business Administration),สถาบันบัณฑิตพัฒนบริหารศาสตร์,NaN,NaN,คณะบริหารธุรกิจ สถาบันบัณฑิตพัฒนบริหารศาสตร์ แ...,02-727-3975,NaN,
104,รองศาสตราจารย์,ดร.,กฤตินี,ณัฏฐวุฒิสิทธิ์,Ph.D. (การตลาด),จุฬาลงกรณ์มหาวิทยาลัย,NaN,NaN,สถาบันบัณฑิตบริหารธุรกิจศศินทร์ จุฬาลงกรณ์มหาว...,NaN,NaN,
105,นาย,,อภิพงศ์,ศานติเกษม,LL.M. (Distinction)\nLL.M. (International Lega...,,NaN,กฎหมายอาญา/ภาษาญี่ปุ่น,สำนักประธานศาลฎีกา สำนักงานศาลยุติธรรม (อาคารศ...,NaN,NaN,
106,ผู้ช่วยศาสตราจารย์,ดร.,วีรวัฒน์,จันทโชติ,Doctor of Science of Law,มหาวิทยาลัยธรรมศาสตร์,NaN,กฎหมายอาญา/กฎหมายแพ่ง/ภาษาญี่ปุ่น,คณะนิติศาสตร์ มหาวิทยาลัยธรรมศาสตร์,NaN,NaN,


บันทึกลงไฟล์ CSV

In [9]:
dfs2.to_csv('Councillist.csv', encoding='utf-8-sig')